## Quaternion PyTorch - Basic mechanisms

In [1]:
import torch
from qtorch import quaternion

### 1 - Quaternion tensors

A quaternion number is represented by:

$$
x = a + bi + cj + dk
$$

where $a$, $b$, $c$, and $d$ are real values, and $i$, $j$, $k$ are the imaginary parts. A `QuaternionTensor` extends the standard PyTorch `tensor` to handle quaternion values, by specifying the real and imaginary components during initialization:

In [30]:
# Simple scalar quaternion
x = quaternion.QuaternionTensor([0.0, 0.3, 0.4, 0.5])
x

tensor([0.0000, 0.3000, 0.4000, 0.5000]) torch.Size([4])


real part: tensor([0.])
imaginary part (i): tensor([0.3000])
imaginary part (i): tensor([0.4000])
imaginary part (j): tensor([0.5000])

In [38]:
# Mini-batch of two scalar quaternions
x = quaternion.QuaternionTensor(torch.rand(2, 4))
print(x)

tensor([[0.0202, 0.8506, 0.9436, 0.8216],
        [0.9868, 0.1789, 0.6938, 0.7060]])


In [39]:
# A vector with 4 quaternions
y = quaternion.QuaternionTensor(torch.rand(16))
print(y)

tensor([0.4077, 0.1442, 0.3995, 0.5595, 0.1937, 0.0787, 0.1166, 0.6930, 0.3232,
        0.8850, 0.6558, 0.2871, 0.2544, 0.9730, 0.5249, 0.5997])


All standard quaternion operations can be applied on the tensor (see `QuaternionTensor` for a full list):

In [41]:
# Conjugation
print(x.conj)

tensor([[ 0.0202, -0.8506, -0.9436, -0.8216],
        [ 0.9868, -0.1789, -0.6938, -0.7060]])


In [42]:
# Element-wise norm
print(x.norm)

tensor([[1.5130],
        [1.4092]])


In [43]:
# Element-wise angle
print(x.theta)

tensor([[1.5574],
        [0.7950]])


In [44]:
# Quaternion multiplication (Hamilton product)
print(x * x)

tensor([[-2.2885,  0.0344,  0.0382,  0.0333],
        [-0.0380,  0.3530,  1.3694,  1.3935]])


In [45]:
# Quaternion matrix multiplication
print(x.t() @ x)

tensor([[0.9743, 0.1937, 0.7038, 0.7134],
        [0.1937, 0.7555, 0.9267, 0.8252],
        [0.7038, 0.9267, 1.3717, 1.2651],
        [0.7134, 0.8252, 1.2651, 1.1736]])


Importantly, quaternion tensors and real-valued tensors are interoperable (real-valued tensors being casted to quaternion tensors with 0 imaginary parts):

In [46]:
# Quaternion scalar multiplication
print(x * torch.rand(2))

tensor([[0.0021, 0.0867, 0.0962, 0.0838],
        [0.2680, 0.0486, 0.1884, 0.1917]])


### 2 - Quaternion gradients

Gradients can be computed with the PyTorch autograd mechanisms:

In [47]:
x = torch.nn.Parameter(quaternion.QuaternionTensor(torch.rand(2, 4)))
y = x.norm().sum()
y.backward()

In [48]:
print(x.grad)

tensor([[0.1852, 0.3120, 0.2404, 0.2355],
        [0.3199, 0.3562, 0.5394, 0.4847]])


### 3 - Quaternion-valued layers

We also provide a number of quaternion-valued layers to implement quaternion neural networks:

In [49]:
from torch import nn
from qtorch.layers import QLinear

In [50]:
model = nn.Sequential(
    QLinear(4, 20),
    nn.ReLU(),
    QLinear(20, 1)
)

In [51]:
x = quaternion.QuaternionTensor(torch.rand(2, 16))
print(model(x))

tensor([[ 1.4971, -0.3514, -1.1493,  0.3259],
        [ 1.4303, -0.2878, -0.6018,  0.0444]], grad_fn=<MmBackward>)


In [52]:
model(x).shape

torch.Size([2, 4])

We also provide layers to easily integrate quaternion-valued and real-valued blocks:

In [56]:
from qtorch.layers import QuaternionToReal
from torch.nn import Softmax

In [57]:
model = nn.Sequential(
    QLinear(4, 10),
    QuaternionToReal(10), # Take the absolute value
    Softmax()
)

In [58]:
model(x).shape

torch.Size([2, 10])